In [69]:
#Importing necessary libraries and frameworks
import tensorflow as tf
import scipy as sc
import numpy as np
import time
import sklearn.preprocessing
from scipy.signal import butter,lfilter


In [70]:
#Defining a butterworth filtter bandpass
def butter_bandpass(lowcut,highcut,fs,order=5):
    #Nyquist rate fs=2*fm
    #fm=0.5*fs
    nyq=0.5*fs
    lowcut=lowcut/nyq
    highcut=highcut/nyq
    #b,a=numerator and denominator values of the IIR Filter
    b,a=butter(order,[lowcut,highcut],btype='band')
    return b,a

In [71]:
#Defining one hot encoding
#This function is used to transfer one column label to one hot label
def one_hot(y_):
    #Function to encode output labels from number indexes
    #e.g [[5],[3],[0]] --> [[0,0,0,0,0,1],[0,0,0,1,0,0],[0,0,0,0,0,0]]
#     y_=y_.reshape((len(y_)))
    y_=np.array(y_)
    y_=y_.reshape(len(y_))
    n_values=np.max(y_)+1
    return np.eye(n_values)[np.array(y_,dtype=np.int32)]

In [72]:
#Checking
a1=[[5],[3],[2]]
a2=one_hot(a1)
print(a2)

[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [73]:
def butter_bandpass_filter(data,lowcut,highcut,fs,order=5):
    b,a=butter_bandpass(lowcut,highcut,fs,order=order)
    y=lfilter(b,a,data)
    return y
    

Defining some variables used to load and process the EEG Data
1. **len_sample** = Length of each sample ( in seconds )
2. **full**= total number of samples per subject 
3. **n_fea**=number of features ( EEG Channels )
4. **label0-label7** = Defines the labels for each sample

In [74]:
len_sample=1
full=7000
len_a=int(full/len_sample)
label0=np.zeros(len_a)
label1=np.ones(len_a)
label2=np.ones(len_a)*2
label3=np.ones(len_a)*3
label4=np.ones(len_a)*4
label5=np.ones(len_a)*5
label6=np.ones(len_a)*6
label7=np.ones(len_a)*7
label=np.hstack((label0,label1,label2,label3,label4,label5,label6,label7))
label=label.T
label.shape=(len(label),1)
print(label)

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [75]:
start1=time.process_time()

In [76]:
feature=sc.io.loadmat("EID-S.mat")
all=feature['eeg_close_8sub_1file']

In [77]:
n_fea=14
all=all[0:full*8,0:n_fea]

### Next steps - Performing a filtering operation on a EEG Dataset to extract the delta wave pattern using a Butterworth filter

1. **start2**= measures the starting time using `time.process_time()` function and stores it in a variable
2. Initialize an empty list **data_f1** to store the filtered EEG data
3. Loop over the columns of the `all` array using `range(all.shape[1])`
4. For each column, applies a bandpass filter using the butter_bandpass_filter function with the specified lowcut, highcut and sampling frequency values. The filtered data is then stored in the `data_f1 list`.
5. Convert the `data_f1` list to a numpy array and then transpose it
6. Replace the original `all` array with the filtered data array `data_f1`
7. Measure the ending time using the `time.process_time()` function and store it in a variable `time3`
8. Print the shape of the filtered data array and the elapsed timem for the filtering operation

In [78]:
#Timing the process
start=time.process_time()

#Empty list
data_f1=[]
for i in range(all.shape[1]):
    x=all[:,i]
    fs=128.0
    lowcut=0.5
    highcut=4.0
    y=butter_bandpass_filter(x,lowcut,highcut,fs,order=3)
    data_f1.append(y)

In [79]:
data_f1=np.array(data_f1)
data_f1=data_f1.T
print('data_f1.shape',data_f1.shape)

data_f1.shape (56000, 14)


In [82]:
all=data_f1
end=time.process_time()
print('PD time',start-end)
all=np.hstack((all,label))
print(all.shape)

PD time 0.0
(56000, 15)


In [83]:
#all=combine data
data_size=all.shape[0]

feature_all=all[:,0:n_fea]
print(all[:,n_fea:n_fea+1])

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [85]:
feature_all=feature_all-4200 #minus Direct Current
#z-score scaling
feature_all=sklearn.preprocessing.scale(feature_all)